<a href="https://colab.research.google.com/github/amiller411/uniwork/blob/big_data_cw3/Copy_of_COM739_17952_CW3_run_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initial Setup**

1. First, you will setup your CoLab environment. Run the cell below.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Download both anime.csv and rating.csv, and store it in your google drive. It is advisable to create a separate project folder, where you can store this dataset and also your code.

The script will give you the id of the two files in your drive.

In [3]:
file_list = drive.ListFile({'q': "'1Oi8cMnAfJVZH9-FyXGxwOrGGCIkkB7uy' in parents"}).GetList()
for f in file_list:
  print('title: %s, id: %s' % (f['title'], f['id']))

title: rating.csv, id: 1f76dQZxRB1fNaReBv_DnUDVkIXNm7mw9
title: anime.csv, id: 1TppJoj4QVJlc_HML20xmH847Brrw0Zfc


If you executed the cells below, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
# Change the id, if it differs from the one below.
id='1TppJoj4QVJlc_HML20xmH847Brrw0Zfc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('anime.csv')

id='1f76dQZxRB1fNaReBv_DnUDVkIXNm7mw9'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('rating.csv')

Here is a list of packages that might be useful to you. 

**Student Activity: Add the packages you need to carry out your analysis here** 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Student Activity: Add your packages here.
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
# from pyspark.sql.functions import col, size
# from pyspark.sql.functions import isnan, when, count, col

**This step initializes the Spark context.**

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a local runtime).

In [7]:
spark

## **From this point onwards, you are supposed to do the coding yourself. Follow the steps as mentioned below in its appropriate place.**

**1. Student Activity: Read the datasets here. You must write the script for the first question and explore both the files here.**

Q1. Identify and describe the number of columns in the two dataset files.

In [7]:
df_anime = spark.read.csv("anime.csv", header=True, inferSchema = True)
df_rat = spark.read.csv("rating.csv", header=True, inferSchema = True)

In [9]:
df_anime.printSchema()
df_rat.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



The Anime dataset contains 7 columns (anime_id, name, genre, type, episodes, rating and members) with anime_id and members being the only integer types, rating the only double type and the remaining types all strings.

**2. Student Activity: Preprocess the datasets here. You must write the script for the second question here. Make sure to check if the script is running is correctly or not**

Q2. Merge/Join/Combine the two datasets and identify the key common column that would you have performed? 

Key column = rating, full join

In [11]:
df_anime.show()

+--------+--------------------+--------------------+-----+--------+------+-------+
|anime_id|                name|               genre| type|episodes|rating|members|
+--------+--------------------+--------------------+-----+--------+------+-------+
|   32281|      Kimi no Na wa.|Drama, Romance, S...|Movie|       1|  9.37| 200630|
|    5114|Fullmetal Alchemi...|Action, Adventure...|   TV|      64|  9.26| 793665|
|   28977|            Gintama°|Action, Comedy, H...|   TV|      51|  9.25| 114262|
|    9253|         Steins;Gate|    Sci-Fi, Thriller|   TV|      24|  9.17| 673572|
|    9969|       Gintama&#039;|Action, Comedy, H...|   TV|      51|  9.16| 151266|
|   32935|Haikyuu!!: Karasu...|Comedy, Drama, Sc...|   TV|      10|  9.15|  93351|
|   11061|Hunter x Hunter (...|Action, Adventure...|   TV|     148|  9.13| 425855|
|     820|Ginga Eiyuu Densetsu|Drama, Military, ...|  OVA|     110|  9.11|  80679|
|   15335|Gintama Movie: Ka...|Action, Comedy, H...|Movie|       1|   9.1|  72534|
|   

In [37]:
df_anime.describe().show()

+-------+------------------+-----------------+------+-----+------------------+-----------------+-----------------+
|summary|          anime_id|             name| genre| type|          episodes|           rating|          members|
+-------+------------------+-----------------+------+-----+------------------+-----------------+-----------------+
|  count|             12294|            12294| 12232|12269|             12294|            12064|            12294|
|   mean|14058.221652838783|         166778.5|  null| null|12.382549774134182|6.473901690981445|18071.33886448674|
| stddev|11455.294700988177|330891.6746051493|  null| null| 46.86535196440979|1.026746306898068|54820.67692490701|
|    min|                 1|    &quot;0&quot;|Action|Movie|                 1|             1.67|                5|
|    max|             34527|                ◯|  Yaoi|   TV|           Unknown|             10.0|          1013917|
+-------+------------------+-----------------+------+-----+------------------+--

In [36]:
df_rat.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           user_id|         anime_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|           7813737|          7813737|           7813737|
|   mean|36727.956744640884|8909.072104295294| 6.144029546937656|
| stddev|20997.946118973723| 8883.94963588107|3.7278004201098067|
|    min|                 1|                1|                -1|
|    max|             73516|            34519|                10|
+-------+------------------+-----------------+------------------+



In [9]:
df_rat.select([count(when(isnan('rating'),True))]).show()

+--------------------------------------------+
|count(CASE WHEN isnan(rating) THEN true END)|
+--------------------------------------------+
|                                           0|
+--------------------------------------------+



In [10]:
df_anime.select([count(when(isnan('rating'),True))]).show()

+--------------------------------------------+
|count(CASE WHEN isnan(rating) THEN true END)|
+--------------------------------------------+
|                                           0|
+--------------------------------------------+



In [8]:
# dataframe.filter(dataframe.ID!='4').show()
df_rating_dropped = df_rat.filter(df_rat.rating!='-1')
# filter null values in rating
# df_rating_dropped = df_rating_dropped.na.drop()

In [12]:
df_rating_dropped.count()

6337241

In [33]:
df_rating_dropped.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           user_id|         anime_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|           6337241|          6337241|           6337241|
|   mean| 36747.91443421514|8902.866382547232| 7.808496946857473|
| stddev|21013.403086875314| 8881.99964731458|1.5724960304974132|
|    min|                 1|                1|                 1|
|    max|             73516|            34475|                10|
+-------+------------------+-----------------+------------------+



In [22]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [13]:
pd_ratings = df_rat.toPandas()

In [14]:
pd_ratings.shape

(7813737, 3)

In [15]:
# df = df[df.line_race != 0]
pd_ratings_dropped = pd_ratings[pd_ratings.rating != -1]

In [16]:
pd_ratings_dropped.shape

(6337241, 3)

In [ ]:
# df_rat = df_rat.selectExpr("user_id as user_id","anime_id as anime_id_rat", "rating as rating_rat")

In [ ]:
# df_rat_renamed = df_rat.selectExpr("user_id as user_id","anime_id_rat as anime_id", "rating_rat as rating_rat")

In [9]:
df_rating_dropped = df_rating_dropped.selectExpr("user_id as user_id","anime_id as anime_id_rating", "rating as rating_rating")

In [13]:
df_anime.count()

12294

In [14]:
df_rating_dropped.count()

6337241

In [15]:
6337241+12294

6349535

In [11]:
# i need a full outer join on anime_id with anime_id and ratings renamed so no ambigious errors
# empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"fullouter") 
outer_join_df = df_anime.join(df_rating_dropped, df_anime.anime_id == df_rating_dropped.anime_id_rating, "fullouter")

In [12]:
outer_join_df.count()

6339609

In [16]:
# this is wrong is gives me 332m rows
# joined_df_on_rating = df_anime.join(df_rating_dropped, df_anime.rating == df_rating_dropped.rating, how='full')

In [ ]:
# i want a join or merge that adds the ratings cols together for each anime_id
# user_anime_df = user_anime_df.join(anime_df, 'anime_id', how='left')
# i think i need a left join on anime_id col
# left_join_df = user_anime_df.join(anime_df, 'anime_id', how='left')

In [17]:
joined_df_on_rating.count()

332052950

In [13]:
outer_join_df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- anime_id_rating: integer (nullable = true)
 |-- rating_rating: integer (nullable = true)



In [25]:
outer_join_df.count()

6339609

In [26]:
outer_join_df_filtered = outer_join_df.filter(outer_join_df.anime_id. isNotNull())
# df.filter(df.col_X. isNotNull()) 

In [27]:
outer_join_df_filtered.count()

6339607

In [14]:
outer_join_df.show()

+--------+------------+--------------------+----+--------+------+-------+-------+---------------+-------------+
|anime_id|        name|               genre|type|episodes|rating|members|user_id|anime_id_rating|rating_rating|
+--------+------------+--------------------+----+--------+------+-------+-------+---------------+-------------+
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     19|              1|           10|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     21|              1|            9|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     23|              1|            9|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     32|              1|           10|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     34|              1|            7|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     43|              1|       

In [ ]:
joined_rating.show()

+--------+--------------------+--------------------+-------+--------+------+-------+-------+--------+------+
|anime_id|                name|               genre|   type|episodes|rating|members|user_id|anime_id|rating|
+--------+--------------------+--------------------+-------+--------+------+-------+-------+--------+------+
|   24259|Mechano: Scientif...|Comedy, Dementia,...|    OVA|       3|  2.55|    120|   null|    null|  null|
|   33860|Saru Kani Gassen ...|          Historical|    ONA|       1|  2.84|     38|   null|    null|  null|
|   23677|Manga Edo Erobanashi|Comedy, Hentai, H...|    OVA|       5|  2.86|    111|   null|    null|  null|
|   31268|       Miira no Yume|            Dementia|  Movie|       1|  2.97|     98|   null|    null|  null|
|   29972|    Hakuchuu Meikyuu|            Dementia|    ONA|       1|  3.02|     99|   null|    null|  null|
|   16786|Bulsajo Robot Pho...|Action, Mecha, Sh...|  Movie|       1|  3.11|    133|   null|    null|  null|
|   32834|Gra-P &am

In [ ]:
# > df.filter(df["age"]>24).show()
# joined_df.filter(joined_df["rating_rat"]>0).show()

In [ ]:
# joined_df.summary().show()`
# joined_df.select("anime_id", "rating_rat").summary().show()
# df.select("age", "name").summary("count").show()

Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [ ]:
# orders_with_product_names = orders.join(products, orders.product_id == products.product_id)
# grouped_orders = orders_with_product_names.groupBy('order_id').agg({'product_name': 'collect_list'}).toDF('order_id', 'product_names')
# grouped_orders = joined_df.groupBy('anime_id').agg({'rating': 'collect_list'}).toDF('anime_id', 'rating')

**3. Student Activity: Now do some exploratory analysis. You must write the script for the third and fourth question here. Make sure to check if the script is running is correctly or not**

Q3. Find the top 10 anime based on user rating. Use tabular/graphical presentation to provide evidence of your analysis.

Q4. Find the top 10 genre based on user rating. Use tabular/graphical presentation to provide evidence of your analysis.

In [28]:
# cast types of two ratings and add the contents of the columns together
outer_join_df = outer_join_df_filtered.withColumn("rating_rating", outer_join_df_filtered.rating_rating.cast('double'))

In [ ]:
i wonder am i better getting all values from each dataframe first
if i can groupby and collect two dataframes then I can concat the 2 cols (ratings and rating_rating) then i can perform the op on them
then sort by desc
https://www.datasciencemadesimple.com/concatenate-two-columns-in-pyspark/#:~:text=Concatenating%20columns%20in%20pyspark%20is%20accomplished%20using%20concat()%20Function.,-1&text=Concatenating%20two%20columns%20is%20accomplished%20using%20concat()%20Function.&text=Concatenating%20multiple%20columns%20is%20accomplished%20using%20concat()%20Function.

In [18]:
outer_join_df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- anime_id_rating: integer (nullable = true)
 |-- rating_rating: double (nullable = true)



In [55]:
# df.groupBy($"name")
  # .agg(collect_list(struct($"food", $"price")).as("foods"))
  # .show(false)
# outer_join_df_filtered.groupBy("anime_id").agg(collect_list(struct("rating", "rating_rating")).as("all_ratings")).show(false)
collected_ratings = outer_join_df_filtered.groupBy("anime_id").agg(collect_list(struct("rating", "rating_rating")))

In [65]:
# df.groupBy("name").agg( *[F.array_join(F.collect_list(column), "").alias(column) for column in df.columns if column !='name' ]).show()
col_list = ["rating", "rating_rating"]
# both_rating_cols = outer_join_df_filtered.groupBy("anime_id").agg( *[F.array_join(F.collect_list(column), "").alias(column) for column in col_list if column !='anime_id' ])
both_rating_cols_2 = outer_join_df_filtered.groupBy("anime_id").agg( *[F.collect_list(column).alias(column) for column in col_list if column !='anime_id' ])

In [66]:
both_rating_cols_2.show()

+--------+--------------------+--------------------+
|anime_id|              rating|       rating_rating|
+--------+--------------------+--------------------+
|       1|[8.82, 8.82, 8.82...|[10.0, 9.0, 9.0, ...|
|       5|[8.4, 8.4, 8.4, 8...|[9.0, 9.0, 10.0, ...|
|       6|[8.32, 8.32, 8.32...|[8.0, 7.0, 10.0, ...|
|      15|[8.08, 8.08, 8.08...|[6.0, 7.0, 9.0, 7...|
|      16|[8.18, 8.18, 8.18...|[9.0, 9.0, 8.0, 9...|
|      17|[7.74, 7.74, 7.74...|[6.0, 10.0, 9.0, ...|
|      19|[8.72, 8.72, 8.72...|[10.0, 7.0, 10.0,...|
|      20|[7.81, 7.81, 7.81...|[8.0, 6.0, 8.0, 9...|
|      22|[8.04, 8.04, 8.04...|[5.0, 7.0, 9.0, 7...|
|      26|[7.78, 7.78, 7.78...|[7.0, 9.0, 5.0, 9...|
|      27|[7.44, 7.44, 7.44...|[9.0, 8.0, 8.0, 6...|
|      28|[8.05, 8.05, 8.05...|[10.0, 9.0, 7.0, ...|
|      31|[7.51, 7.51, 7.51...|[9.0, 9.0, 6.0, 9...|
|      43|[8.34, 8.34, 8.34...|[7.0, 8.0, 8.0, 1...|
|      44|[8.83, 8.83, 8.83...|[7.0, 10.0, 10.0,...|
|      47|[8.15, 8.15, 8.15...|[8.0, 8.0, 7.0,

In [64]:
both_rating_cols.show()

+--------+--------------------+--------------------+
|anime_id|              rating|       rating_rating|
+--------+--------------------+--------------------+
|       1|8.828.828.828.828...|10.09.09.010.07.0...|
|       5|8.48.48.48.48.48....|9.09.010.09.09.09...|
|       6|8.328.328.328.328...|8.07.010.09.09.09...|
|      15|8.088.088.088.088...|6.07.09.07.09.07....|
|      16|8.188.188.188.188...|9.09.08.09.09.08....|
|      17|7.747.747.747.747...|6.010.09.09.010.0...|
|      19|8.728.728.728.728...|10.07.010.010.08....|
|      20|7.817.817.817.817...|8.06.08.09.09.06....|
|      22|8.048.048.048.048...|5.07.09.07.07.08....|
|      26|7.787.787.787.787...|7.09.05.09.07.07....|
|      27|7.447.447.447.447...|9.08.08.06.07.05....|
|      28|8.058.058.058.058...|10.09.07.08.08.08...|
|      31|7.517.517.517.517...|9.09.06.09.010.08...|
|      43|8.348.348.348.348...|7.08.08.010.09.08...|
|      44|8.838.838.838.838...|7.010.010.09.07.0...|
|      47|8.158.158.158.158...|8.08.07.07.09.0

In [68]:
both_rating_cols.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- rating: string (nullable = false)
 |-- rating_rating: string (nullable = false)



In [67]:
both_rating_cols_2.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- rating: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- rating_rating: array (nullable = false)
 |    |-- element: double (containsNull = false)



at long last lol now lets concat these two cols

In [76]:
ratings = both_rating_cols_2.select(concat(col("rating"), col("rating_rating")))

In [84]:
ratings.printSchema()

root
 |-- concat(rating, rating_rating): array (nullable = false)
 |    |-- element: double (containsNull = false)



In [83]:
all_ratings = both_rating_cols_2.withColumn("all_ratings", concat(col("rating"), col("rating_rating")))
# df.select(concat(col("tokens"), col("tokens_bigrams"))).show(truncate=False) 

In [85]:
all_ratings.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- rating: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- rating_rating: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- all_ratings: array (nullable = false)
 |    |-- element: double (containsNull = false)



In [ ]:
df = df.select(
  'name',
  F.expr('AGGREGATE(scores, 0, (acc, x) -> acc + x)').alias('Total')
)

In [96]:
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

array_mean = udf(lambda x: float(np.mean(x)), FloatType())
# df.select(array_mean("longitude").alias("avg")).show()

In [97]:
all_ratings.select(array_mean("all_ratings").alias("avg")).show()

+---------+
|      avg|
+---------+
| 8.844716|
| 8.419862|
| 8.369697|
|  8.18985|
| 8.272468|
|7.8917804|
| 8.820652|
| 7.851944|
| 8.121358|
|7.8494983|
|7.5230494|
| 8.151735|
|7.5254235|
| 8.322972|
|  8.85006|
| 8.147625|
|7.2513185|
| 7.390013|
|7.3444815|
|7.2541227|
+---------+
only showing top 20 rows



In [98]:
all_ratings_avg = all_ratings.withColumn("avg_rating", array_mean("all_ratings"))

In [99]:
all_ratings_avg.show()

+--------+--------------------+--------------------+--------------------+----------+
|anime_id|              rating|       rating_rating|         all_ratings|avg_rating|
+--------+--------------------+--------------------+--------------------+----------+
|       1|[8.82, 8.82, 8.82...|[10.0, 9.0, 9.0, ...|[8.82, 8.82, 8.82...|  8.844716|
|       5|[8.4, 8.4, 8.4, 8...|[9.0, 9.0, 10.0, ...|[8.4, 8.4, 8.4, 8...|  8.419862|
|       6|[8.32, 8.32, 8.32...|[8.0, 7.0, 10.0, ...|[8.32, 8.32, 8.32...|  8.369697|
|      15|[8.08, 8.08, 8.08...|[6.0, 7.0, 9.0, 7...|[8.08, 8.08, 8.08...|   8.18985|
|      16|[8.18, 8.18, 8.18...|[9.0, 9.0, 8.0, 9...|[8.18, 8.18, 8.18...|  8.272468|
|      17|[7.74, 7.74, 7.74...|[6.0, 10.0, 9.0, ...|[7.74, 7.74, 7.74...| 7.8917804|
|      19|[8.72, 8.72, 8.72...|[10.0, 7.0, 10.0,...|[8.72, 8.72, 8.72...|  8.820652|
|      20|[7.81, 7.81, 7.81...|[8.0, 6.0, 8.0, 9...|[7.81, 7.81, 7.81...|  7.851944|
|      22|[8.04, 8.04, 8.04...|[5.0, 7.0, 9.0, 7...|[8.04, 8.04, 

In [102]:
all_ratings_avg.count()

12294

In [103]:
df = all_ratings_avg.na.drop(subset=["avg_rating"])

In [105]:
df.count()

12066

In [106]:
# df.sort(df.department.asc(),df.state.desc()).show(truncate=False)
# df.sort(col("department").asc(),col("state").desc()).show(truncate=False)
df.sort(col("avg_rating").desc()).show()

+--------+--------------------+--------------------+--------------------+----------+
|anime_id|              rating|       rating_rating|         all_ratings|avg_rating|
+--------+--------------------+--------------------+--------------------+----------+
|   33662|              [10.0]|                  []|              [10.0]|      10.0|
|   30120|               [9.6]|                  []|               [9.6]|       9.6|
|   23005|               [9.5]|                  []|               [9.5]|       9.5|
|   32281|[9.37, 9.37, 9.37...|[9.0, 5.0, 8.0, 1...|[9.37, 9.37, 9.37...|  9.398156|
|   28977|[9.25, 9.25, 9.25...|[10.0, 10.0, 7.0,...|[9.25, 9.25, 9.25...|  9.349748|
|   33607|              [9.33]|                  []|              [9.33]|      9.33|
|    5114|[9.26, 9.26, 9.26...|[10.0, 10.0, 9.0,...|[9.26, 9.26, 9.26...|   9.29137|
|   26313|              [9.25]|                  []|              [9.25]|      9.25|
|     820|[9.11, 9.11, 9.11...|[10.0, 10.0, 10.0...|[9.11, 9.11, 

In [86]:
all_ratings.describe().show()

+-------+------------------+
|summary|          anime_id|
+-------+------------------+
|  count|             12294|
|   mean|14058.221652838783|
| stddev|11455.294700988166|
|    min|                 1|
|    max|             34527|
+-------+------------------+



In [57]:
collected_ratings.show()

+--------+-------------------------------------------+
|anime_id|collect_list(struct(rating, rating_rating))|
+--------+-------------------------------------------+
|       1|                       [{8.82, 10.0}, {8...|
|       5|                       [{8.4, 9.0}, {8.4...|
|       6|                       [{8.32, 8.0}, {8....|
|      15|                       [{8.08, 6.0}, {8....|
|      16|                       [{8.18, 9.0}, {8....|
|      17|                       [{7.74, 6.0}, {7....|
|      19|                       [{8.72, 10.0}, {8...|
|      20|                       [{7.81, 8.0}, {7....|
|      22|                       [{8.04, 5.0}, {8....|
|      26|                       [{7.78, 7.0}, {7....|
|      27|                       [{7.44, 9.0}, {7....|
|      28|                       [{8.05, 10.0}, {8...|
|      31|                       [{7.51, 9.0}, {7....|
|      43|                       [{8.34, 7.0}, {8....|
|      44|                       [{8.83, 7.0}, {8....|
|      47|

In [29]:
grouped_ratings = outer_join_df_filtered.groupBy('anime_id').agg({'rating': 'collect_list'}).toDF('anime_id', 'rating')

In [30]:
grouped_ratings.show()

+--------+--------------------+
|anime_id|              rating|
+--------+--------------------+
|       1|[8.82, 8.82, 8.82...|
|       5|[8.4, 8.4, 8.4, 8...|
|       6|[8.32, 8.32, 8.32...|
|      15|[8.08, 8.08, 8.08...|
|      16|[8.18, 8.18, 8.18...|
|      17|[7.74, 7.74, 7.74...|
|      19|[8.72, 8.72, 8.72...|
|      20|[7.81, 7.81, 7.81...|
|      22|[8.04, 8.04, 8.04...|
|      26|[7.78, 7.78, 7.78...|
|      27|[7.44, 7.44, 7.44...|
|      28|[8.05, 8.05, 8.05...|
|      31|[7.51, 7.51, 7.51...|
|      43|[8.34, 8.34, 8.34...|
|      44|[8.83, 8.83, 8.83...|
|      47|[8.15, 8.15, 8.15...|
|      48|[7.14, 7.14, 7.14...|
|      52|[7.32, 7.32, 7.32...|
|      53|[7.24, 7.24, 7.24...|
|      54|[7.21, 7.21, 7.21...|
+--------+--------------------+
only showing top 20 rows



In [31]:
grouped_ratings_ratings = outer_join_df_filtered.groupBy('anime_id').agg({'rating_rating': 'collect_list'}).toDF('anime_id', 'rating_rating')

In [32]:
grouped_ratings_ratings.show()

+--------+--------------------+
|anime_id|       rating_rating|
+--------+--------------------+
|       1|[10.0, 9.0, 9.0, ...|
|       5|[9.0, 9.0, 10.0, ...|
|       6|[8.0, 7.0, 10.0, ...|
|      15|[6.0, 7.0, 9.0, 7...|
|      16|[9.0, 9.0, 8.0, 9...|
|      17|[6.0, 10.0, 9.0, ...|
|      19|[10.0, 7.0, 10.0,...|
|      20|[8.0, 6.0, 8.0, 9...|
|      22|[5.0, 7.0, 9.0, 7...|
|      26|[7.0, 9.0, 5.0, 9...|
|      27|[9.0, 8.0, 8.0, 6...|
|      28|[10.0, 9.0, 7.0, ...|
|      31|[9.0, 9.0, 6.0, 9...|
|      43|[7.0, 8.0, 8.0, 1...|
|      44|[7.0, 10.0, 10.0,...|
|      47|[8.0, 8.0, 7.0, 7...|
|      48|[9.0, 6.0, 9.0, 8...|
|      52|[7.0, 8.0, 6.0, 9...|
|      53|[6.0, 7.0, 7.0, 8...|
|      54|[5.0, 8.0, 8.0, 8...|
+--------+--------------------+
only showing top 20 rows



In [108]:
top_ten_anime_rating = grouped_ratings_ratings.withColumn("avg_rating", array_mean("rating_rating"))

In [110]:
df_top_ten_anime_user_rating = top_ten_anime_rating.na.drop(subset=["avg_rating"])

In [113]:
# df.where(size(col("tokens")) <= 3).show()
df_anime_filtered = df_top_ten_anime_user_rating.where(size(col("rating_rating")) > 10)

In [115]:
df_anime_filtered.count()

7240

In [119]:
df_anime.distinct().count()
# df.distinct().count())

12294

In [120]:
df_anime_nullless = df_anime.na.drop(subset=["anime_id"])

In [121]:
df_anime_nullless.count()

12294

In [114]:
df_anime_filtered.sort(col("avg_rating").desc()).show()

+--------+--------------------+----------+
|anime_id|       rating_rating|avg_rating|
+--------+--------------------+----------+
|   28977|[10.0, 10.0, 7.0,...|  9.449495|
|   32281|[9.0, 5.0, 8.0, 1...|  9.426313|
|     820|[10.0, 10.0, 10.0...|  9.389789|
|    5114|[10.0, 10.0, 9.0,...| 9.3227415|
|    9969|[6.0, 9.0, 7.0, 8...| 9.2725525|
|    9253|[9.0, 7.0, 10.0, ...|  9.261326|
|     918|[9.0, 7.0, 9.0, 1...|  9.236398|
|   11061|[9.0, 9.0, 8.0, 1...|  9.234586|
|   15417|[6.0, 8.0, 10.0, ...|  9.202258|
|   15335|[6.0, 10.0, 10.0,...|   9.19143|
|   32935|[10.0, 9.0, 10.0,...|  9.171484|
|    4181|[9.0, 8.0, 9.0, 9...|  9.165292|
|    2904|[9.0, 9.0, 10.0, ...|  9.059837|
|   28851|[10.0, 8.0, 10.0,...|  9.029411|
|   23273|[8.0, 8.0, 8.0, 1...|   8.98404|
|     263|[7.0, 9.0, 10.0, ...|  8.954365|
|   30276|[9.0, 8.0, 9.0, 8...|   8.93968|
|    2001|[10.0, 9.0, 10.0,...|  8.933176|
|   12355|[9.0, 9.0, 10.0, ...|  8.932257|
|   24701|[6.0, 9.0, 9.0, 1...|  8.930297|
+--------+-

In [107]:
# sort for genre

so above we have all the ratings for each anime, need to rank them, should first look at how many ratings each anime has and whether it should be included

In [ ]:
# df.withColumn("column<2", expr("aggregate(filter(column, x -> x < 2), 0D, (x, acc) -> acc + x)")) 
# https://coderedirect.com/questions/280859/sum-of-array-elements-depending-on-value-condition-pyspark

now to work out most popular genre

**4. Student Activity: Design the recommendation system. Remember to split the dataset into training and testing to validate your recommendation model. This section would help you in answering question 5**

Q5. Design a collaborative filter-based recommendation system. 

**Student Activity: Analyse the output of the test dataset here.**

Q6. Give example of best three anime recommendations for minimum of 10 users.

In [ ]:
spark.stop()